In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[ 0.0191, -0.0645,  0.1349, -0.0088, -0.1100,  0.0480, -0.2438, -0.1700,
         -0.1191, -0.2213],
        [ 0.0802,  0.0070,  0.2331,  0.0445, -0.0705,  0.0711, -0.2024, -0.0649,
         -0.1602,  0.1925]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.1281, -0.1272, -0.2462, -0.2875,  0.2344,  0.2438,  0.1672,  0.0192,
          0.0379,  0.2774],
        [ 0.0906, -0.0745, -0.1915, -0.4249,  0.1710,  0.1911,  0.1966,  0.0607,
          0.0878,  0.2495]], grad_fn=<AddmmBackward0>)

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X
        

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1673,  0.0405, -0.1417, -0.1023,  0.0533, -0.1537, -0.0490,  0.3420,
          0.1585,  0.0912],
        [ 0.1428,  0.0911, -0.0442, -0.1455,  0.0983, -0.0885, -0.1903,  0.1889,
          0.2734,  0.2004]], grad_fn=<AddmmBackward0>)

In [3]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [6]:
net = FixedHiddenMLP()
print(net,'\n')

FixedHiddenMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
) 



In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))

In [8]:
net = NestMLP()
print(net)

NestMLP(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=32, out_features=16, bias=True)
)
